In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import branca.colormap as cm
import folium
import fiona
import shapefile
import rasterstats
import rasterio as rio
import math
import json
import os
import matplotlib.pyplot as plt
import seaborn as sns

from folium.plugins import MarkerCluster
from folium.features import ColorLine
from folium.features import GeoJson
from shapely.geometry import mapping
from shapely.geometry import LineString
from rasterio.plot import show
from shapely.geometry import Polygon, mapping
from rasterio.mask import mask
from geopy.distance import geodesic

In [2]:
# reading shape files and mapping the route 
def read_shape(shapefile, route):
    fp = shapefile
    dat = gpd.read_file(fp)
    da = dat[dat['ROUTE_NUM'] == route]
    
    # da.to_file("da.shp")
    # points = folium.GeoJson(da.to_json())
    # map = folium.Map(location = [47.6, -122.3], zoom_start = 12).add_child(points)
    
    return(da)
    
# reading raster files and extract elevation 
def extract_elev(shapefile, route, rasterfile):
    da = read_shape(shapefile, route)
    raw_elev = pd.DataFrame(rasterstats.gen_point_query(da, rasterfile))
    elev = [x for x in raw_elev.iloc[0,:] if x is not None] 
    print('The number of points: {}' .format(len(elev)))
    elev_meter = np.array(elev) * 0.3048
    return(elev_meter)


# extracting points from linestring 
def extract_point(shapefile, route):
    da = read_shape(shapefile, route)
    geoms = da.geometry.values # list of shapely geometries
    geometry = geoms[0] # shapely geometry
    geoms = [mapping(geoms[0])]
    ge = np.array(geoms[0]['coordinates'])[:,[0,1]]

    coord = []
    for i in range(len(ge)):
            coord.append([ge[i][1], ge[i][0]])

    return(coord)


# map points 
def map_point(shapefile, route):
    coord = extract_point(shapefile, route)
    loca = pd.DataFrame(np.array(coord)).values.tolist()

    map = folium.Map(location=[47.6, -122.3], zoom_start=12)
    for point in range(0, len(loca)):
        folium.Marker(loca[point]).add_to(map)
    
    return(map)


# distance measure 
def dist_measure(shapefile, route):
    coord = extract_point(shapefile, route)
    dis = []
    for i in range(len(coord)-1):
        dis.append(geodesic(coord[i],coord[i+1]).m)

    dist = np.insert(np.cumsum(dis), 0, 0)
    
    return(dis, dist)

# route plots 
def gradient(shapefile, route, rasterfile):
    dis, dist = dist_measure(shapefile, route)
    elev = extract_elev(shapefile, route, rasterfile)
    stress = np.insert(abs(np.diff(elev)/ dis), 0, 0)
    normalized_stress = sum(stress)
    
    print('Normalized stress is {}' .format(normalized_stress))
    return(elev, stress, normalized_stress)


def profile_plot(shapefile, route, rasterfile):
    elev, stress, normalized_stress = gradient(shapefile, route, rasterfile)
    fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(15, 8))
    
    ax[0].plot(dist, elev, label= 'elevation', color='b', linewidth=4)
    ax[0].set_ylabel('Elevation (meter)', color='b')
    ax[0].tick_params('y', colors='b')
    ax[0].legend()
    ax[0].grid()
    
    # ax[1] = ax.twinx()
    ax[1].plot(dist, stress, label= 'stress index', color='r')
    ax[1].set_xlabel('Plain distance (meter)')
    ax[1].set_ylabel('Stress index', color='r')
    ax[1].tick_params('y', colors='r')
    ax[1].legend()
    ax[1].grid()
    
    return()

In [3]:
def make_lines(gdf, gradient, idx, geometry = 'geometry'):   
    coordinate_1 = gdf.loc[idx]['coordinates']
    coordinate_2 = gdf.loc[idx + 1]['coordinates']
    line = LineString([coordinate_1, coordinate_2])
    
    data = {'gradient': gradient,
            'geometry':[line]}
    df_line = pd.DataFrame(data, columns = ['gradient', 'geometry'])
    
    return df_line

In [4]:
def route_to_shp(shapefile, route_num, rasterfile):
    """input the number of route, then output a GeoDataFrame with gradient and geometry
    information of that route, and elevation_gradient for each line segment. 
    Also will save the route as shapefile named 'route_num'."""
    routes_shp = gpd.read_file(shapefile)
    route = routes_shp[routes_shp['ROUTE_NUM'] == route_num]
    
    elevation = rasterstats.point_query(route, rasterfile)
    _, elevation_gradient,_ = gradient(shapefile, route, rasterfile)
    
    route_geometry = route.geometry.values # list of shapely geometries
    route_geometry = [mapping(route_geometry[0])]
    coordinates_route = route_geometry[0]['coordinates']
    linestring_route = []
    for i in range(len(coordinates_route)):
        linestring_route.append(coordinates_route[i][:2])
        linestring_route_df = pd.DataFrame()  
        linestring_route_df['coordinates'] = linestring_route
        
    df_route = pd.DataFrame(columns = ['gradient', 'geometry'])
    for idx in range(len(linestring_route_df) - 1):
        df_linestring = make_lines(linestring_route_df, elevation_gradient[idx], idx)
        df_route = pd.concat([df_route, df_linestring])
    
    gdf_route = gpd.GeoDataFrame(df_route)
    gdf_route.to_file("route_%d.shp"%route_num)
    return gdf_route

In [5]:
def route_map(route_num):
    """Visualize route_num map according to gradient. 
    e.g. route_num = 45 , shows plot of route_45."""
    gdf_route = gpd.read_file("route_%d.shp"%route_num)
    UW_coords = [47.655548, -122.303200]
    figure_size = folium.Figure(height = 400)
    route_map = folium.Map(location = UW_coords, zoom_start = 12)
    min_grade = min(gdf_route['gradient'])
    max_grade = max(gdf_route['gradient'])
    route_json = gdf_route.to_json()
    linear_map = cm.linear.Paired_06.scale(min_grade, max_grade )
    route_layer = folium.GeoJson(route_json, style_function = lambda feature: {
        'color': linear_map(feature['properties']['gradient']),
        'weight': 7})
    route_layer.add_child
    route_map.add_child(linear_map)
    route_map.add_child(route_layer)
    route_map.add_to(figure_size)
    return route_map

In [7]:
shapefile = 'six_routes.shp'
rasterfile = 'seattle_dtm.tif'
route_to_shp(shapefile,45,rasterfile)

AttributeError: 'float' object has no attribute '__geo_interface__'